In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from keras.models import load_model

augmentation_path="D:/University_Work/y2/sem2/AI/code/Project/Font/augmentation"
save_path="D:/University_Work/y2/sem2/AI/code/Project/model"
accs=[]
for count,folder in enumerate(os.listdir(augmentation_path)):
    print(str(count)+":"+folder)
    if count<2:
        continue
    data=[]
    for file in os.listdir(augmentation_path+"/"+folder):
        # print(file)
        if file.endswith(".png"):
            img=cv2.imread(augmentation_path+"/"+folder+"/"+file)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            if img.shape != (100,100):
                img=cv2.resize(img,(100,100))
            data.append([img,file[:-4].split("_")[0]])
        else:
            print("No.pngfilefound")
    df=pd.DataFrame(data,columns=['image','fontname'])
    X=df['image']
    y=df['fontname']
    y = y.replace(['AngsanaNew', 'Chonburi', 'iannnnn-DUCK', 'MN KHAIPHALO',
         'pluempluem', 'pphometowntest', 'THBaijam', 'THCharmofAU',
         'THFahkwang', 'THKodchasal', 'THKoHo', 'THKrub', 'THMaliGrade6',
         'THSarabunNew'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

    # change to numpy array
    X_train = np.array(X_train)
    X_train = np.array([np.array(item) for item in X_train])
    X_test = np.array(X_test)
    X_test = np.array([np.array(item) for item in X_test])
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # Scale images to the [0, 1] range
    X_train= X_train.astype('float32') / 255 
    X_test= X_test.astype('float32') / 255

    # Scale labels to categorical data
    y_train = keras.utils.to_categorical(y_train)
    y_test = keras.utils.to_categorical(y_test)
    X_train = X_train.reshape((X_train.shape[0], 100, 100, 1))
    X_test = X_test.reshape((X_test.shape[0], 100, 100, 1))

    model = keras.models.Sequential()

    model.add(layers.Conv2D(10,(5,5),activation='relu', input_shape=(100,100,1)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(20, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())

    model.add(layers.Dense(14, activation='relu'))
    model.add(layers.Dense(14, activation='softmax'))

    batch_size = 128
    epochs = 20

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    print('Loss: ', test_loss)
    accs.append(test_acc)
    # folder is the name of the model
    model.save(save_path+'/'+folder+'.h5')

print(accs)



0:uni0E01
1:uni0E02
2:uni0E03
Epoch 1/20
108/108 [==============================] - 11s 35ms/step - loss: 0.2217 - accuracy: 0.9351 - val_loss: 2.8286e-05 - val_accuracy: 1.0000
Epoch 2/20
108/108 [==============================] - 3s 27ms/step - loss: 3.9462e-05 - accuracy: 1.0000 - val_loss: 1.5200e-05 - val_accuracy: 1.0000
Epoch 3/20
108/108 [==============================] - 3s 27ms/step - loss: 2.7818e-05 - accuracy: 1.0000 - val_loss: 1.0285e-05 - val_accuracy: 1.0000
Epoch 4/20
108/108 [==============================] - 3s 27ms/step - loss: 1.7541e-05 - accuracy: 1.0000 - val_loss: 8.1864e-06 - val_accuracy: 1.0000
Epoch 5/20
108/108 [==============================] - 3s 27ms/step - loss: 1.4392e-05 - accuracy: 1.0000 - val_loss: 6.2516e-06 - val_accuracy: 1.0000
Epoch 6/20
108/108 [==============================] - 3s 27ms/step - loss: 1.2789e-05 - accuracy: 1.0000 - val_loss: 4.9494e-06 - val_accuracy: 1.0000
Epoch 7/20
108/108 [==============================] - 3s 27ms/step 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.